## Imports

In [3]:
import math
from funciones import *

In [2]:
h = 0.15
z_om_w = 0.12*h

# 3.1. u_200: Wind speed at a blending height assumed to be 200 m (eq 32)
# filtrado_ws: Velocidad del viento medido por estación meteorológica a las 10:30 [Kh/hr]
blending_h = 200 
u_w = 1.8*1000/3600 # [m/s]
z_x = 2.9  # altura del anemómetro, varía de 2 a 2.80m (relativo a cada estación)
u_200 = u_w*math.log(blending_h/z_om_w)/math.log(z_x/z_om_w) # [m/s]  
u_200

0.9165218524652456

In [1]:
# ------------------------------------------------
# Importando librerías utilizadas
# ------------------------------------------------

import ee          # Earth Engine Python API
ee.Initialize()

import os
import numpy as np
import math
import pandas as pd   

import geemap
import geemap.colormaps as cmp    # Paletas para visualización de imágenes

from geemap import cartoee        # Elaboración de gráficas
import cartopy.crs as ccrs        # Importar sistema de coordenadas para gráficas 

from pprint import pprint
import matplotlib.pyplot as plt

# Funciones en archivo Funciones.py
from funciones import *

C:\Users\USUARIO\Documents\00-notebooks-2022\funciones.py:477: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if lai_method is 0:
C:\Users\USUARIO\Documents\00-notebooks-2022\funciones.py:481: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if lai_method is 1:
C:\Users\USUARIO\Documents\00-notebooks-2022\funciones.py:486: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if lai_method is 2:
C:\Users\USUARIO\Documents\00-notebooks-2022\funciones.py:798: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if filtrado is 'cold':
C:\Users\USUARIO\Documents\00-notebooks-2022\funciones.py:816: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if filtrado is 'cold':


---

In [48]:
# ------------------------------------------------------------------------------------------------
# Pruebas: En caso de realizar alguna edición al archivo 'funciones.py' realizar lo siguiente:
# ------------------------------------------------------------------------------------------------

import importlib
import sys

importlib.reload(sys.modules['funciones']) # Importa las funciones del archivo con las modificaciones aplicadas
from funciones import *

In [33]:
# 1. Condiciones estables

u_200 = 18.3154164577919
z_om = 0.035434332 # 0.005 # 0.035434332

pixel_elev = 232.551712
pixel_ts = 290.869873

Kc = 1.05
Inst_ETr = 0.43 # mm/hr

u_star = get_u_star(u_200, z_om)
r_ah = get_rah(u_star)
air_dens = get_air_dens(pixel_elev, 20+273.15)

lambda_LE = ( 2.501-0.00236*(pixel_ts-273.15) ) * 10**6 # Eq 53
LE = Inst_ETr * Kc * lambda_LE / 3600  # [W m-2]

print(u_star, r_ah, air_dens, lambda_LE, LE)

0.8692961853185864 8.405264172603264 1.1601300527484308 2459181.09972 308.42229625655


In [50]:
pix_values_f = {
    'Ts_k': 290.869873,
    'Ts': 290.869873 - 273.15,
    'R_n': 529.7671931,
    'G':60.68105961,
    'Elev_m': 232.551712,
    'LAI': 1.968574,
    'Slope_d': 0
}

pix_values_fpros = procesado_inicial(pix_values_f, 34.92, 0.43, 1.05)
pix_values_fpros

{'Ts': 290.869873,
 'Ts_dem': 292.362609128,
 'Z_om': 0.035434332,
 'u_200': 17.780523937825766,
 'u*': 0.8439088277210368,
 'r_ah': 8.658120216102656,
 'air_dens_p': 1.1601300527484308,
 'R_n': 529.7671931,
 'G': 60.68105961,
 'LE': 308.42229625655,
 'H0': 160.66383723345,
 'dT0': 1.1942667933421582}

In [51]:
pix_values_c = {
    'Ts_k': 314.259521,
    'Ts': 314.259521 - 273.15,
    'R_n': 601.9251246,
    'G': 124.5408483,
    'Elev_m': 231.477097,
    'LAI': 0.022141,
    'Slope_d': 0
}

pix_values_cpros = procesado_inicial(pix_values_c, 34.92, 0.43, Kc=0)
pix_values_cpros

{'Ts': 314.259521,
 'Ts_dem': 315.7452721305,
 'Z_om': 0.005,
 'u_200': 17.780523937825766,
 'u*': 0.6879556574447105,
 'r_ah': 10.620835809357704,
 'air_dens_p': 1.1602762152961477,
 'R_n': 601.9251246,
 'G': 124.5408483,
 'LE': 0.0,
 'H0': 477.3842763,
 'dT0': 4.352429076997993}

In [52]:
pix_f_procesado, pix_c_procesado = get_H(pix_values_fpros, pix_values_cpros)

In [56]:
u_200 = 17.780523937825766

In [53]:
pix_f_procesado

{'Ts': 290.869873,
 'Ts_dem': 292.362609128,
 'Z_om': 0.035434332,
 'u_200': 17.780523937825766,
 'u*': 0.8439088277210368,
 'r_ah': 8.658120216102656,
 'air_dens_p': 1.1601300527484308,
 'R_n': 529.7671931,
 'G': 60.68105961,
 'LE': 308.42229625655,
 'H0': 160.66383723345,
 'dT0': 1.1942667933421582,
 'dT1': 1.194266793342159,
 'H1': 160.66383723345012,
 'a_coef': 0.13506426891232076,
 'b_coef': -38.293475265829755}

In [57]:
pix_f_stability_cor = stability_corr(pix_f_procesado)
pix_c_stability_cor = stability_corr(pix_c_procesado)

In [59]:
pix_f_iter = iteracion(u_200, pix_f_procesado, pix_values_f, pix_f_stability_cor, pix_f_procesado['H0'])
pix_c_iter = iteracion(u_200, pix_c_procesado, pix_values_c, pix_c_stability_cor, pix_c_procesado['H0'])

In [60]:
pix_f_iter

{'Ts': 290.869873,
 'Ts_dem': 292.362609128,
 'Z_om': 0.035434332,
 'u*': 0.9417907978108201,
 'r_ah': 7.638071647299355,
 'air_dens_p': 0.6042584438461526,
 'dT0': 2.022764962793954}

---

In [15]:
for elev in [2.5, 2.8, 2.9, 3]:
    print(290.869873 + 6.5*(233-elev)/1000) # K

292.36812299999997
292.366173
292.365523
292.364873


In [17]:
##########################
# 2. Emisividad e broadband
##########################
# 2.1. Z_om : Momentum roughness length 
pixel_lai = 1.968574 #pixel_values['LAI']
pixel_slope = 0 #pixel_values['Slope_d']
# Flat model momemtum roughnes length
if pixel_lai < 0.2778:
    Z_om_flat = 0.005 # m, represents roughness typical of bare agricultural soils
else:
    Z_om_flat = 0.018*pixel_lai

# Adjusting momentm roughness length for slopes (Z_om_mtn)
if pixel_slope < 5:
    Z_om = Z_om_flat
else:
    Z_om = Z_om_flat * ( 1+(pixel_slope-5)/20 )

Z_om_flat

0.035434332

0.035434332

In [12]:
# 3. u_star: Friction velocity
# z_om_w: Roughness length for the weather station surface (Brutsaert, 1982) eq 40 en manual
# z_om_w = 0.12*h     # h: Average vegetation height around the weather station 
# h = altura del grass de la estación met. (relativo a cada estación)
h = 0.15
z_om_w = 0.12*h

# u_200: Wind speed at a blending height assumed to be 200 m (eq 32)
blending_h = 200

# dato_ws: Velocidad del viento medido por estación meteorológica a las 10:30 
dato_ws = 1.6 
u_w = dato_ws*1000/3600 # dato_ws en Km/hr -> m/s
# z_x = 2.5  # altura del anemómetro, varía de 2 a 2.80m (relativo a cada estación)
for z_x in [2.5, 2.6, 2.7, 2.8, 2.9, 3]:
    u_200 = u_w*math.log(blending_h/z_om_w)/math.log(z_x/z_om_w) # [m/s]    
    # u_200 = 18.3154164577919
    # u*1: Friction velocity (eq 31)
    u_star = get_u_star(u_200, 0.035434332) # Z_om = 0.035434332
    
    print(f'z_x: {z_x} | u_200: {u_200} | u_star: {u_star}')

z_x: 2.5 | u_200: 0.8391943394177256 | u_star: 0.03983029485995805
z_x: 2.6 | u_200: 0.8325756996333659 | u_star: 0.03951615740478192
z_x: 2.7 | u_200: 0.8263047024183553 | u_star: 0.039218519948941655
z_x: 2.8 | u_200: 0.82035052411455 | u_star: 0.038935919523331204
z_x: 2.9 | u_200: 0.8146860910802183 | u_star: 0.03866707114414831
z_x: 3 | u_200: 0.8092875091338538 | u_star: 0.03841084073284865


In [13]:
1000*101.3*( (293.15-0.0065*232.551712)/293.15)**5.26/1.01/293.15/287

# =1000*101.3*(((293.15-0.0065*$E$7)/293.15)^5.26)/1.01/293/287

1.1601300527484300

1.160130052748431

In [16]:
6.5*(233-2.5)/1000 

1.49825

---

In [46]:
pixel_elev = 232.551712
pixel_ts = 290.869873 # °C

##########################
# 1. Ts_dem
Tlapse_rate = 6.5   # C/km
Elev_station = 2.5  # m
Ts_dem = pixel_ts + Tlapse_rate/1000*(pixel_elev-Elev_station) # K

19.79135695485659

In [57]:
# Selección de parámetros
pixel_elev = 232.551712
pixel_ts = 290.869873 # K

##########################
# 1. Ts_dem
Tlapse_rate = 6.5   # C/km
Elev_station = 2.5  # m
Ts_dem = pixel_ts + Tlapse_rate/1000*(pixel_elev-Elev_station) # K
    

##########################
# 2. Emisividad e broadband
# 2.1. Z_om : Momentum roughness length 
pixel_lai = 1.968574 # pixel_values['LAI']
pixel_slope = 0 # pixel_values['Slope_d']

# Flat model momemtum roughnes length
if pixel_lai < 0.2778:
    Z_om_flat = 0.005 # m
    # represents roughness typical of bare agricultural soils
else:
    Z_om_flat = 0.018*pixel_lai

# Adjusting momentm roughness length for slopes (Z_om_mtn)
if pixel_slope < 5:
    Z_om = Z_om_flat
else:
    Z_om = Z_om_flat * ( 1+(pixel_slope-5)/20 )

# Resultado: Z_om

##########################
# 3. u_star: Friction velocity
# z_om_w: Roughness length for the weather station surface (Brutsaert, 1982) eq 40 en manual
# z_om_w = 0.12*h     # h: Average vegetation height around the weather station 
# siendo h la altura del grass de la estación met (relativo a cada estación)
h = 0.15
z_om_w = 0.12*h

# 3.1. u_200: Wind speed at a blending height assumed to be 200 m (eq 32)
# filtrado_ws: Velocidad del viento medido por estación meteorológica a las 10:30 [Kh/hr]
# blending_h = 200 
# u_w = filtrado_ws*1000/3600 # [m/s]
# z_x = 2.5  # altura del anemómetro, varía de 2 a 2.80m (relativo a cada estación)
u_200 = 18.3154164577919 # u_w*math.log(blending_h/z_om_w)/math.log(z_x/z_om_w) # [m/s]    
    
# 3.2. u*1: Friction velocity (eq 31)
u_star = get_u_star(u_200, Z_om)

##########################
# 4. r_ah1: Aerodynamic resistance (eq 30) [s/m]
r_ah = get_rah(u_star)

##########################
# 5. Air density p (eq 37) For 20 °C: 293° = 273° + 20°
air_dens_p = get_air_dens(pixel_elev, air_temp_K=20+273.15)

##########################
# 6. dT (eq 28)
# resultado1 = get_dT(pixel_values, filtrado_ETr, Kc, r_ah, air_dens_p)
# A continuación se debe obtener los coeficientes a y b

print(u_star, r_ah, air_dens_p)

0.8692961853185864 8.405264172603264 1.1601300527484308


1.1601300527484308

In [56]:
1000*101.3*((293.15-0.0065*pixel_elev)/293.15)**5.26/(1.01*293.15*287)

1.1601300527484308

---

In [30]:
# Prueba
u_200 = 18.3154164577919
et_inst = 0.43
elev = 232.551712
ts = 290.869873
Rn = 529.7671931
G = 60.68105961
z_om = 0.035434332
ts_dem = ts + 6.5/1000*(elev - 2.5) # 292.365209128

In [39]:
u_star = get_u_star(u_200, z_om)
r_ah = get_rah(u_star)
print(u_star, r_ah)

0.8692961853185864 8.405264172603264


In [44]:
dT1 = 0.131160627926246*ts_dem+(-37.1905326238396)
p1 =1000*101.3*((293.15-0.0065*elev)/293.15)**5.26/(1.01*293*287)
print(dT1, p1)

1.1562717891771

1.1562717891771044 1.1607239759836263


1.1562717891771

In [27]:
math.log(2/0.1)/(0.869296185318584*0.41)

8.405264172603289

-----

In [2]:
# ------------------------------------------------
# Importando áreas de estudio
# ------------------------------------------------

# Importar asset público: Distrito de Ferreñafe
asset_dist = ee.FeatureCollection('users/CesarVilca/distritos') 
dist_fcol = asset_dist.filterMetadata('DISTRITO', 'equals', 'FERREÑAFE')
roi = dist_fcol.geometry().bounds()   # ee.Geometry tipo caja 

# Importar shapefile de Áreas voladas (archivo en local)
ruta_ferrenafe = r'input/Ferreñafe_areas_2.shp'  

# Lectura como FeatureCollection con geemap
predios_agricolas = geemap.shp_to_ee(ruta_ferrenafe)
predios_vis = ee.Image().paint(predios_agricolas, 2, 2)  # Ver solo bordes, no relleno

predios_bound = predios_agricolas.geometry().bounds()
predios_bound_vis = ee.Image().paint(predios_bound, 1, 1)  # Ver solo bordes, no relleno

type(predios_agricolas) # Comprobación

# ------------------------------------------------
# Parámetros de visualización de mapas con geemap
# ------------------------------------------------

# Para imágenes Landsat 8 y 9 TOA
vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

# Paletas de colores
albedo_cmap = ['68181f', 'd6604d', 'f9dbc7', 'd1e5f0', '2166ac', '053061']  # 0 - 0.5
ndvi_cmap = ['ffffff', 'ce7e45', 'fad163', '74a909', '3a7405', '1a3b03']  # -0.2 - 1  '0048fb',
ts_cmap = ['fefccc', 'fbd976', 'f08c39', 'ea4a33', 'e43d32', '812026']  # 280 - 330 en K o 6.85 - 56.85 en °C
rn_cmp = ['560e0d', 'fd5903', 'fbfe04', '3aff01', '01feef', '0000fe', 'fd6dfd', 'ffffff']
rn_cmp_r = ['ffffff', 'fd6dfd', '0000fe', '01feef', '3aff01', 'fbfe04', 'fd5903', '560e0d']

# -------------------------------------------------------
# Parámetros de visualización de gráficas con matplotlib
# -------------------------------------------------------

plt.rcParams['font.family'] = 'serif'  #'serif', 'Arial'  # default es 'sans-serif'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['figure.dpi'] = 72

In [4]:
# =============================================================
# Inputs iniciales
# =============================================================

# 2 de Diciembre 2021: Almácigo
# 4 de Enero 2022: Transplante

# 8 Imágenes seleccionadas
lista_ids = [
    # 'LANDSAT/LC09/C02/T1/LC09_010065_20211212', # 0 - 12/12 - l9
    'LANDSAT/LC09/C02/T1/LC09_010065_20220113',  # 1 - 01/13 - l9 OJO: NDVI negativos -> Posible riego 01 13
    'LANDSAT/LC09/C02/T1/LC09_010065_20220129',  # 2 - 01/29 - l9
    'LANDSAT/LC08/C02/T1/LC08_010065_20220310',  # 3 - 03/10 - l8
    # 'LANDSAT/LC09/C02/T1/LC09_010065_20220403', # 4 - 04/03 - l9
    # 'LANDSAT/LC09/C02/T1/LC09_010065_20220521', # 5 - 05/21 - l9
    # 'LANDSAT/LC08/C02/T1/LC08_010065_20220529', # 6 - 05/29 - l8
    'LANDSAT/LC08/C02/T1/LC08_010065_20220614'  # 7 - 06/14 - l8
]

# Importar DEM SRTM
# Ojo: El mapa Aspect no se puede obtener en los límites -> usar áreas mayores al área de estudio
dem = ee.Image("USGS/SRTMGL1_003").clip(roi)

# =============================================================
# Procesar las imagenes e incluirlos en una lista
# =============================================================

lista_imgprocesadas = []

for img_id in lista_ids:
    
    # Procesar la imagen a TOA
    img_ee = ee.Image(img_id).clip(roi)
    img_toa = convert_RAW_to_TOA(img_ee) 

    # Obtener fechas 
    img_date = img_ee.date()
    fecha = img_date.format('YYYY-MM-dd').getInfo()

    # Procesar Radiación Neta (produce un dict)
    lai_method = 2  # 0: Savi con l=0.1, 1: Savi con L=0.5, 2: Relación NDVI - IAF 
    img_procesada = getRadiacionNeta(img_ee, roi, dem, lai_method=lai_method) # roi: Ferreñafe

    print(f'Productos y Rn de imagen {fecha} procesada!')

    # Agregar datos al dict
    img_procesada['fecha'] = fecha
    img_procesada['img_productos'] = img_procesada['img_productos'].clip(predios_agricolas)
    img_procesada['img_toa'] = img_toa
    
    # Agregar a lista en blanco
    lista_imgprocesadas.append(img_procesada)

Productos y Rn de imagen 2022-01-13 procesada!
Productos y Rn de imagen 2022-01-29 procesada!
Productos y Rn de imagen 2022-03-10 procesada!
Productos y Rn de imagen 2022-06-14 procesada!


## Desarrollo H

In [5]:
# Selección de datos
index = 2

img_productos = lista_imgprocesadas[index]['img_productos']
d2 = lista_imgprocesadas[index]['d2']
R_n = lista_imgprocesadas[index]['img_Rn']
fecha = lista_imgprocesadas[index]['fecha']

print(f'Procesando imagen {fecha}')

# Inputs principales
img_productos_clipped = img_productos.clip(predios_agricolas)

img_tempK = img_productos_clipped.select('Ts_k')
img_tempC = img_productos_clipped.select('Ts_c')

img_albedo = img_productos_clipped.select('albedo')
img_ndvi = img_productos_clipped.select('NDVI')
img_lai = img_productos_clipped.select('LAI')

img_slope = img_productos_clipped.select('slope')
dem = ee.Image("USGS/SRTMGL1_003")

Procesando imagen 2022-03-10


In [7]:
# =============================================================
# Valores de pixel frío y caliente
# Selección de datos mediante el'index'
# =============================================================

# Datos de estación: Velocidad de viento y ET
list_weather_st = [
    [1.6, 0.3],
    [1.6, 0.69],
    [3.2, 0.53],
    [1.6, 0.46]
] # km/h

filtrado_ws, filtrado_et = list_weather_st[index]

# Pixel frío
list_coords_pixf = [
    [-79.78527, -6.598056], # 0 '2022-01-13' n1: 50 n2: 10
    [-79.78472, -6.598345], # 1 '2022-01-29' n1: 50 n2: 10
    [-79.78445, -6.596984], # 2 '2022-03-10' n1: 50 n2: 5
    [-79.778737, -6.604828] # 3 '2022-06-14' n1: 50 n2: 5
]

pixf_coord = list_coords_pixf[index]
pixf_ee_coord = ee.Geometry.Point(pixf_coord)

# Pixel caliente
pix_c_values = get_pixel_values(pixf_ee_coord, d2, img_productos)

list_coords_pixc = [
    [-79.78198, -6.602693],  # 0 '2022-01-13' n1: 40 n2: 97 
    [-79.778464, -6.605114], # 1 '2022-01-29' n1: 50 n2: 70
    [-79.777924, -6.604045], # 2 '2022-03-10' n1: 3 n2: 80 
    [-79.784729, -6.595629]  # 3 '2022-06-14' n1: 10 n2: 80 
]

pixc_coord = list_coords_pixc[index]
pixc_ee_coord = ee.Geometry.Point(pixc_coord)

In [8]:
##### Procesado Inicial #####
# 1. Ts_dem
img_tsdem = img_tempK.expression('img_ts + tlapse_rate/1000*(dem-Elev_station)',
                                 {'img_ts':img_tempK,
                                  'tlapse_rate':6.5,
                                  'dem': dem,
                                  'Elev_station': 3})

# 2. Momentum roughness length Z_om
# Flat model momemtum roughnes length
# img_lai < 0.2778 -> Z_om = 0.005 # represents roughness typical of bare agricultural soils
# img_lai >= 0.2778 -> Z_om = 0.018*img_lai
img_zom1 = img_lai.where(img_lai.gte(0.2778), img_lai.multiply(0.018)).where(img_lai.lt(0.2778), 0.005).rename('Z_om')

# 3. Friction velocity
h = 0.15
z_om_w = 0.12*h
blending_h = 200
u_w = filtrado_ws*1000/3600 # dato_ws en Km/hr -> m/s
z_x = 2.5  # altura del anemómetro, varía de 2 a 2.80m (relativo a cada estación)
u_200 = u_w*math.log(blending_h/z_om_w)/math.log(z_x/z_om_w) # [m/s]

# u*1: Friction velocity (eq 31)
img_u_star = get_u_star_sp(u_200, img_zom1)
# get_stats(img_u_star, predios_agricolas, 30)

img_r_ah = get_rah_sp(img_u_star)
# get_stats(img_r_ah, predios_agricolas, 30)

img_air_dens_p = get_air_dens_sp(dem, air_temp_K=20+273.15) # Para 20°
# get_stats(img_air_dens_p, predios_agricolas, 30)

img_lambdaLE = img_tempC.expression('( 2.501-0.002361*(Ts_c) )*10**6',{'Ts_c':img_tempC})
# Kc = 1.05 # Kc = 0 para pixel caliente 
# Inst_ETr * Kc * lambda_LE / 3600 # Eq 53

Inst_ETr = filtrado_et
Kc = 0
img_LE = img_lambdaLE.expression('Inst_ETr*Kc*lambda_LE/3600',
                             {'Inst_ETr': Inst_ETr, 'Kc':Kc, 'lambda_LE': img_lambdaLE})

img_Rn = R_n.select('R_n').clip(predios_agricolas).rename('0_Rn')
img_G = img_tempK.expression('(Ts_K - 273.15)*(0.0038 + 0.0074*albedo)*(1-0.98*NDVI**4)',
                             {'Ts_K': img_tempK,
                              'albedo': img_albedo,
                              'NDVI': img_ndvi}).multiply(R_n.select('R_n')).rename('1_G')

img_H = img_Rn.subtract(img_G).subtract(img_LE).rename('H')
img_H_stats = get_stats(img_H, predios_agricolas, 30)
img_H_stats
# H = R_n - G - LE 

NameError: name 'get_u_star_sp' is not defined

## Otro

In [ ]:
# =============================================================
# Inputs iniciales
# =============================================================

index = 0 # Imagen Landsat: 0 1 Enero, 2 Marzo, 3 Junio 
lai_method = 2  # 0: Savi con l=0.1, 1: Savi con L=0.5, 2: Relación NDVI - IAF 

# En caso de guardar los archivos
save_files = True
ruta_base = os.getcwd() # A esta se anexará
carpeta_output = 'output2'

lista_imgprocesadas_ET = []

for index in range(4):

    # Selección de datos
    img_productos = lista_imgprocesadas[index]['img_productos']
    d2 = lista_imgprocesadas[index]['d2']
    R_n = lista_imgprocesadas[index]['img_Rn']
    fecha = lista_imgprocesadas[index]['fecha']

    print(f'Procesando imagen {fecha}')
    
    # Generar estadísticas y guardar en DataFrame
    img_productos_dict = get_stats(img_productos, predios_agricolas, 30)
    img_productos_df = pd.DataFrame.from_dict(img_productos_dict, orient='index')

    # =============================================================
    # Valores de pixel frío y caliente
    # Selección de datos mediante el'index'
    # =============================================================

    # Datos de estación: Velocidad de viento y ET
    list_weather_st = [
        [1.6, 0.3],
        [1.6, 0.69],
        [3.2, 0.53],
        [1.6, 0.46]
    ] # km/h

    filtrado_ws, filtrado_et = list_weather_st[index]
    
    # Pixel frío
    list_coords_pixf = [
        [-79.78527, -6.598056], # 0 '2022-01-13' n1: 50 n2: 10
        [-79.78472, -6.598345], # 1 '2022-01-29' n1: 50 n2: 10
        [-79.78445, -6.596984], # 2 '2022-03-10' n1: 50 n2: 5
        [-79.778737, -6.604828] # 3 '2022-06-14' n1: 50 n2: 5
    ]

    pixf_coord = list_coords_pixf[index]
    pixf_ee_coord = ee.Geometry.Point(pixf_coord)

    # Pixel caliente
    pix_c_values = get_pixel_values(pixf_ee_coord, d2, img_productos)

    list_coords_pixc = [
        [-79.78198, -6.602693],  # 0 '2022-01-13' n1: 40 n2: 97 
        [-79.778464, -6.605114], # 1 '2022-01-29' n1: 50 n2: 70
        [-79.777924, -6.604045], # 2 '2022-03-10' n1: 3 n2: 80 
        [-79.784729, -6.595629]  # 3 '2022-06-14' n1: 10 n2: 80 
    ]

    pixc_coord = list_coords_pixc[index]
    pixc_ee_coord = ee.Geometry.Point(pixc_coord)

    pix_h_values = get_pixel_values(pixc_ee_coord, d2, img_productos)

    # =============================================================
    # Flujo de calor sensible H - Proceso Iterativo
    # =============================================================

    n_iteraciones = 40 # <-- Establecer cantidad de iteraciones

    iteracion_output = parte_iterativa(n_iteraciones, 
                                       pix_c_values, pix_h_values, 
                                       filtrado_ws, filtrado_et)

    # Resultados del proceso iterativo
    dict_pix_f_resultados = iteracion_output['dict_pix_f_resultados']
    dict_pix_c_resultados = iteracion_output['dict_pix_c_resultados']
    resultados_coef = iteracion_output['list_coef_resultados']
    u_200 = iteracion_output['u_200']
    list_stability_result = iteracion_output['list_stability_resultados']

    # Coeficientes a y b
    coefs_a = resultados_coef[0] 
    coefs_b = resultados_coef[1]
    print('\nCoeficientes:')
    print(f'\ta_coef: {coefs_a[-1]}')
    print(f'\tb_coef: {coefs_b[-1]}')

    # ==============================================================
    # - Gráfico 1: Pixel frío y caliente durante el proceso iterativo

    # Plot Pixel Frío
    # get_grafica_iteracion(fecha, dict_pix_f_resultados, 'Pixel Frío', 'blue', n_iteraciones, save_fig=save_files)
    # plt.close()

    # Plot Pixel Caliente
    # get_grafica_iteracion(fecha, dict_pix_c_resultados, 'Pixel Caliente', 'red', n_iteraciones, save_fig=save_files)
    # plt.close()

    # ==============================================================
    # - Gráfico 2: Gráfico 1 combinado

    x_values = np.arange(1, n_iteraciones+2+1)

    # Desarrollo de gráfica
    fig, axs = plt.subplots(3,1, figsize=(10,12), layout='constrained', sharex=True)

    title_name = f'{fecha} - Comparativa Pixel Frío y Pixel Caliente'
    axs[0].set_title(title_name, fontsize='14')
    ylabels = ['Rah', 'Air density', 'dT']

    list_variables = ['r_ah', 'air_dens_p', 'dT0']

    for n in range(0, 2+1):

        variable = list_variables[n]
        axs[n].plot(x_values, dict_pix_f_resultados[variable], 'b-', label='Pixel Frío')#, color='blue')
        axs[n].plot(x_values, dict_pix_c_resultados[variable], 'r-', label='Pixel Caliente')#, color='red')

        axs[n].set(xticks=np.arange(0, n_iteraciones+3, step=2), ylabel=ylabels[n])
        axs[n].grid(alpha=0.2)

    axs[0].legend(fontsize='12')
    axs[2].set_xlabel('N° Iteraciones');

    # Guardar
    if save_files is True:
        nombre_img = f'{fecha}_Comparativa-Pixeles.jpg'
        plt.savefig(os.path.join(ruta_base, carpeta_output, nombre_img))

    # ==================================================================
    # - Gráfico 3: Coeficientes de selección de pixeles frío y caliente

    fig, ax = plt.subplots(figsize=(10, 6), layout='constrained')

    ax.plot(x_values, coefs_a, label='Coef a', color='blue')
    ax.plot(x_values, coefs_b, label='Coef b', color='red')

    title_name = f'{fecha} - Coeficientes a y b' 
    ax.set(title=title_name, xlabel='N° Iteraciones', xticks=np.arange(0, n_iteraciones+2+1, step=2))

    texto = f'a: {round(coefs_a[-1], 4)}\nb: {round(coefs_b[-1], 4)}'
    ax.text(0.85, 0.5, s=texto, transform=ax.transAxes, fontsize=12)

    ax.legend(fontsize=12, frameon=False, labelspacing=0.2, loc='best')
    ax.grid(alpha=0.2)

    # Guardar
    if save_files is True:
        nombre_img = f'{fecha}_Coeficientes.jpg'
        plt.savefig(os.path.join(ruta_base, carpeta_output, nombre_img)) #, dpi=400);

    # =============================================================
    # Balance de energía: LE = Rn - G - H
    # =============================================================

    # Corrección de H
    img_H = get_H_corregido(img_productos, iteracion_output)
    img_H_stats = get_stats(img_H, predios_agricolas, 30)

    # Componentes de balance de energía
    img_Rn = R_n.select('R_n').clip(predios_agricolas)

    img_tempK = img_productos.select('Ts_k')
    img_G = img_tempK.expression('(Ts_K - 273.15)*(0.0038 + 0.0074*albedo)*(1-0.98*NDVI**4)',
                                 {'Ts_K': img_tempK,
                                  'albedo': img_productos.select('albedo'),
                                  'NDVI': img_productos.select('NDVI')}).multiply(R_n.select('R_n'))

    # Balance de Energía
    img_LE = img_Rn.expression('Rn - G - H',
                               {'Rn': img_Rn,
                                'G': img_G,
                                'H': img_H})
    
    img_LE_gt0 = img_LE.updateMask(img_LE.gt(0))
    
    # Unir los componentes en una sola imagen
    img_final = ee.Image([img_Rn, img_G, img_H, img_LE, img_LE_gt0]).rename(['0_Rn', '1_G', '2_H', '3_LE', '3_LE_vis'])
    img_final_stats = get_stats(img_final, predios_agricolas, 30)
    img_final_stats_df = pd.DataFrame.from_dict(img_final_stats, orient='index', dtype='float') # .round(3)

    # =============================================================
    # Evapotranspiración del cultivo ETinst (mm/hr y mm/d)
    # =============================================================

    # ET (mm/hr)
    img_lambda = img_tempK.expression('(2.501 - 0.00236*(Ts - 273.15))*10**6', {'Ts':img_tempK}).rename('lambda')
    img_ET_inst = img_LE.expression('3600*img_LE/lambda', {'img_LE': img_LE,'lambda': img_lambda}).rename('ET')

    # ET (mm/día)
    img_ET_inst = img_ET_inst.multiply(24) # mm/hr * 24 hrs/dia = mm/dia

    # Generar estadísticas y guardar en dataframe
    img_ET_inst_stats = get_stats(img_ET_inst, predios_agricolas, 30)
    img_ET_inst_stats_df = pd.DataFrame.from_dict(img_ET_inst_stats, orient='index', dtype='float')#.round(3)

    # =============================================================
    # Resultados del procesamiento
    # =============================================================

    # Generar dataframe final
    lista_dfs = [
        img_productos_df[['Ts_c', 'NDVI', 'LAI', 'albedo']], 
        img_final_stats_df, 
        img_ET_inst_stats_df
    ]
    df_stats = pd.concat(lista_dfs, axis=1)

    print('\nResultados:')
    display(df_stats)

    # Guardar dataframe final
    if save_files is True:
        nombre_csv = f'{fecha}_Resultados.csv'
        df_stats.to_csv(os.path.join(ruta_base, carpeta_output, nombre_csv))
    
    # Agregar el resultado al diccionario
    lista_imgprocesadas[index]['img_CompBalance'] = img_final
    lista_imgprocesadas[index]['img_ETinst'] = img_ET_inst
    
    lista_imgprocesadas[index]['pix_frio'] = pix_c_values     # c de cold
    lista_imgprocesadas[index]['pix_caliente'] = pix_h_values # h de hot
    
    # Finalmente agregamos cada imagen procesada a una lista
    lista_imgprocesadas_ET.append(lista_imgprocesadas[index])